# 测试用
## 先删除所有新建的表
## 删除TYPE必须先删除PROCEFURE

In [5]:
DROP TABLE IF EXISTS DwgItem
DROP TABLE IF EXISTS DwgIpart
DROP PROCEDURE IF EXISTS pyproc
DROP TYPE IF EXISTS dwgtype


Commands completed successfully.

Total execution time: 00:00:00.001

# 新建两个表
## DwgItem 用于 一对多对照表
## DwgIpart 用于 Ipart

In [6]:
CREATE TABLE DwgItem
(
    RefItemCode nvarchar(50) NULL,
    ItemCode nvarchar(50) NULL
)

Commands completed successfully.

Total execution time: 00:00:00.015

In [7]:
CREATE TABLE DwgIpart
(
    Ipart nvarchar(50) NULL,
    ItemCode nvarchar(50) NULL
)

Commands completed successfully.

Total execution time: 00:00:00.015

# 表值类型

## 用于存储过程

In [8]:
CREATE TYPE dwgtype AS TABLE (
    Ipart nvarchar(50) NULL,
    ItemCode nvarchar(50) NULL,
    [Name] nvarchar(200) NULL,
    [Path] nvarchar(200) NULL,
    RefCode nvarchar(4000) NULL
)


Commands completed successfully.

Total execution time: 00:00:00.001

# 存储过程

## 更新drawing

In [12]:
CREATE OR ALTER PROCEDURE pyproc
    @tvp dwgtype READONLY
WITH
    ENCRYPTION
AS
BEGIN
    SET NOCOUNT ON
    BEGIN TRANSACTION

    TRUNCATE TABLE drawing
    INSERT INTO drawing
    SELECT *
    FROM @tvp

    UPDATE t1 SET RefCode=t2.RefItemCode FROM drawing t1 INNER JOIN (
SELECT
            ItemCode,
            (
		SELECT RefItemCode+','
            FROM DwgItem
            WHERE ItemCode=t1.ItemCode
            FOR XML PATH('')
	)AS RefItemCode
        FROM DwgItem t1
        GROUP BY ItemCode ) AS t2 ON t1.ItemCode=t2.ItemCode

    UPDATE t1 SET RefCode=t2.RefItemCode FROM drawing t1 INNER JOIN (
SELECT
            Ipart,
            (
	SELECT ItemCode+','
            FROM DwgIpart
            WHERE Ipart=t1.Ipart
            FOR XML PATH('')
	)AS RefItemCode
        FROM DwgIpart t1
        GROUP BY Ipart ) AS t2 ON t1.Ipart=t2.Ipart

    COMMIT TRANSACTION
END


Commands completed successfully.

Total execution time: 00:00:00.003